# Interface of β2 Adrenergic Receptor in Complex with Empty Gs-Protein

In [ ]:
import mdtraj as md
import mdciao

### Download example data and load into the namespace

In [ ]:
import numpy as np
import os
if not os.path.exists("mdciao_example"):
    mdciao.examples.fetch_example_data("b2ar@Gs")
traj = md.load("mdciao_example/traj.xtc",top="mdciao_example/top.pdb")

### Create consensus labeler objects

In [ ]:
GPCR = mdciao.nomenclature.LabelerGPCR("adrb2_human")
CGN = mdciao.nomenclature.LabelerCGN("GNAS2_HUMAN")

### Guess molecular fragments
This would be done anyway by the `mdciao.cli.interface` call in the cell below, here we do it have the fragments defined in the namespace

In [ ]:
fragments = mdciao.fragments.get_fragments(traj.top);
fragment_names = ["Galpha", "Gbeta", "Ggamma","B2AR","P0G"]

### Compute G$\alpha$-B2AR interface
Using the above fragment definitions

In [ ]:
intf = mdciao.cli.interface(traj,
                            title="3SN6 beta2AR-Galpha interface",
                            fragments=fragments, fragment_names = fragment_names,                             
                            interface_selection_1=[0], 
                            interface_selection_2=[3],
                            GPCR_UniProt=GPCR, CGN_UniProt=CGN,                            
                            accept_guess=True, no_disk=True, figures=False)

### Plot each residues's participation in the interface

In [ ]:
ifig = intf.plot_frequency_sums_as_bars(4, title_str = intf.name, 
                                 list_by_interface=True, 
                                 interface_vline=True);
ifig.figure.savefig("intf.svg")

### Plot contact matrix

In [ ]:
ifig, iax = intf.plot_interface_frequency_matrix(4, grid=True, pixelsize=.5);
ifig.savefig("matrix.svg")

### Flareplot
We combine a lot of information into one single flareplot:  

* the molecular topology with sub-fragments and consensus labels, 

* the secondary structure, 

* the individual contact-pairs 

* the participation of each residue in the interface.

In [ ]:
ifig, iax, flareplot_attrs  = intf.plot_freqs_as_flareplot(4, 
                                                           fragments=fragments, fragment_names = fragment_names,  
                                                           scheme="consensus_sparse", consensus_maps=[GPCR, CGN], 
                                                           aura=intf.frequency_sum_per_residue_idx_dict(4,return_array=True),
                                                           SS=True)
ifig.figure.savefig("flare.svg")

### Coarse-Grained Frequencies and Flareplots

In [ ]:
ifig, iax, flareplot_attrs = intf.plot_freqs_as_flareplot(4, 
                                                          fragments=fragments, fragment_names = fragment_names,  
                                                          consensus_maps=[GPCR, CGN],                                         
                                                          coarse_grain=True,
                                                         )
freqs = intf.frequency_as_contact_matrix_CG(4, fragments=fragments, fragment_names = fragment_names,
                                            consensus_labelers=[GPCR, CGN],
                                            interface=True).round(1).replace(0,"")  
freqs

### Grab a representative frame
This frame will be used to plot the interface frequencies as a 3D heatmap (see `frequency_to_bfactor` below).

In [ ]:
repframe = intf.repframes(return_traj=True)[-1]

### Save the interface as a heatmap and view externally

In [ ]:
intf.frequency_to_bfactor(4, pdbfile="interface_heatmap.pdb", 
                          geom=repframe, 
                          interface_sign=True
                         );

### Save all mdciao objects for later reuse
We can save all mdciao objects to numpy .npy (pickle) files and later reload them without having to compute everything again.

In [ ]:
import numpy as np
np.save("GPCR.npy", GPCR)
np.save("CGN.npy",CGN)
np.save("intf.npy",intf)